In [3]:
import gurobipy as gp
# import numpy as np
import math

## Explain the model
As we know the optimization problem is multi-objective optimization problem. But here as you wanted first we implement $z_1$ as single-objective. 
I wrote each part separately and simple you can see the parameters values(fixed values), decision variables, Objective Function, Constraints, Solver and Solution results.

In [4]:
# Define dimensions of optimization problem

I = 3 # number of Customer
J = 5 # Number of Supplier
T = 4 # number of time periods

In [5]:
# Define parameter values
c = [[550, 480, 420, 890, 790],[570, 85, 450, 235, 525],[825, 430, 850, 390, 720]]
p = [[66, 73, 62, 60, 71],[121, 134, 127, 142, 122],[28, 26, 29, 25, 23]]
s0 = [[0.95, 0.97, 0.94, 0.99, 0.98],[0.92, 0.99, 0.91, 0.99, 0.92],[0.99, 0.98, 0.99, 0.97, 0.99]]
f0 = [[0.95, 0.98, 0.92, 0.96, 0.99],[0.89, 0.92, 0.95, 0.90, 0.91],[0.93, 0.87, 0.88, 0.83, 0.87]]
gamma = [[0.0013, 0.0011, 0.001, 0.001, 0.0011],[0.00, 0.0014, 0.002, 0.0015, 0.002],[0.0011, 0.0016, 0.001, 0.002, 0.0011]]
Lambda = [[0.001, 0.002, 0.0015, 0.001, 0.001],[0.00, 0.0015, 0.001, 0.002, 0.002],[0.001, 0.002, 0.0015, 0.0015, 0.0012]]
d = [[454, 540, 675, 755],[327, 320, 290, 285],[645, 650, 637, 663]]
beta = [0.08, 0.1, 0.09, 0.12, 0.09]
o = [45000, 64500, 37250, 53400, 42250]
v = [85, 150, 85, 100, 150]
g = [33500, 55200, 33500, 38400, 55200]
h = [12, 35, 7]
w = [0.75, 0.85, 0.60]
b = [17, 38, 10]
W = 150

In [6]:
# Define the decision variables

x = {} # order quantity for each product in each period
y = {} # binary variable for each product in each period
z = {}
model = gp.Model("Minz1")

for j in range(J):
    for t in range(T):
        for i in range(I):
            x[i,j,t] = model.addVar(vtype=gp.GRB.INTEGER,name="X%d%d%d" % (i+1,j+1,t+1), lb = 0, ub = c[i][j])
        y[j,t] = model.addVar(vtype=gp.GRB.BINARY, name="y%d%d" % (j+1,t+1))
        # z[j,t] = model.addVar(vtype=gp.GRB.CONTINUOUS, name="z%d%d" % (j+1, t+1))

Restricted license - for non-production use only - expires 2024-10-28


## Define Objective function

As we can see the $z_1$ has 4 part, and each part is sum of terms. So first we create each part using <span>quicksum</span> function, and name as <span color="red"> a, b, c, d </span>. Finally we define $z_1$ as sum of $z_{1a}$, $z_{1b}$, $z_{1c}$, $z_{1d}$.

In [7]:
# Define Objective Function (z1)
z1_a = gp.quicksum(p[i][j]*x[i,j,t] for i in range(I) for j in range(J) for t in range(T))
z1_b = gp.quicksum(o[j]*math.exp(-beta[j])*y[j,t] for j in range(J) for t in range(T))
z1_c = gp.quicksum(h[i]*(gp.quicksum(x[i,j,k] for j in range(J) for k in range(t))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*t)*x[i,j,t]) for j in range(J))-gp.quicksum(d[i][k] for k in range(t))) for i in range(I) for t in range(T))
z1_d = gp.quicksum(g[j]*(gp.quicksum(w[i]*x[i,j,t]/v[j] for i in range(I))) for j in range(J) for t in range(T))
z1_obj = z1_a + z1_b + z1_c + z1_d
# model.addGenConstrExp(x, y)
# *math.exp(-beta[j]*gp.quicksum(y[j,k] for k in range(t)))


In [8]:
z2_obj = gp.quicksum(f0[i][j]*math.exp(Lambda[i][j]*t)*x[i,j,t] for i in range(I) for j in range(J) for t in range(T))
z3_obj = gp.quicksum(s0[i][j]*math.exp(gamma[i][j]*t)*x[i,j,t] for i in range(I) for j in range(J) for t in range(T))

In [9]:
# Add z_1 as objective function to model
model.setObjectiveN(z1_obj, index=1, priority=0)
model.setObjectiveN(-z2_obj, index=2, priority=1)
model.setObjectiveN(-z3_obj, index=3, priority=1)
model.ModelSense = gp.GRB.MINIMIZE

## Define constraints
In this part we define the constraints separately and add them as constraint to model. Constraint (6) which has ==, is impossible because $x_{ijt}$ is integer and there is some float values in constraint. So we define that as two constraint to bound the value in $(0, 0.5)$ interval.

In [10]:
# Define Constraints

# Constraint (4)
for i in range(I):
    for t in range(T):
        model.addConstr(gp.quicksum(x[i,j,k] for k in range(t) for j in range(J))-gp.quicksum(d[i][k] for k in range(t)) >= gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*t) for j in range(J)))*x[i,j,t])
# Constraint (5)
for i in range(I):
    for t in range(T):
        for j in range(J):
            model.addConstr(gp.quicksum(d[i][k] for k in range(t,T))*y[j,t]-x[i,j,t] >= 0)
            
# Constraint (6)
for i in range(I):
    model.addConstr(gp.quicksum(x[i,j,t] for t in range(T) for j in range(J))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*T))*x[i,j,T-1] for j in range(J)) - gp.quicksum(d[i][t] for t in range(T)) <= 0.5)
    model.addConstr(gp.quicksum(x[i,j,k] for k in range(T) for j in range(J))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*T))*x[i,j,T-1] for j in range(J)) - gp.quicksum(d[i][t] for t in range(T)) >= 0.0)

# Constraint (7)    
for t in range(T):
    model.addConstr(gp.quicksum(w[i]*(gp.quicksum(x[i,j,k] for k in range(t) for j in range(J))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*t))*x[i,j,t] for j in range(J))-gp.quicksum(d[i][k] for k in range(t))) for i in range(I)) <= W)


## Run Optimization
Here we run the optimization problem and gurobi try to solve that.

In [11]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 82 rows, 80 columns and 492 nonzeros
Model fingerprint: 0xd7a35e7a
Variable types: 0 continuous, 80 integer (20 binary)
Coefficient statistics:
  Matrix range     [4e-03, 3e+03]
  Objective range  [8e-01, 6e+04]
  Bounds range     [1e+00, 9e+02]
  RHS range        [2e+02, 3e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 7 rows and 3 columns
Presolve time: 0.01s
Presolved: 75 rows and 77 columns
------------------------------

## Print the results of optimization

Now we print the solution of Gurobi for minimizing $z_1$, and maximizing $z_2$ and $z_3$. The value of $x_{ijt}$ and $y_{jt}$ are shown below.

In [12]:
for i in range(I):
    for j in range(J):
        for t in range(T):
            print("X%d%d%d = %d"%(i+1, j+1, t+1, x.get((i,j,t)).X), end = "\t")
        print(" ")

X111 = 0	X112 = 0	X113 = 0	X114 = 4	 
X121 = 0	X122 = 0	X123 = 0	X124 = 479	 
X131 = 0	X132 = 0	X133 = 0	X134 = 2	 
X141 = 545	X142 = 0	X143 = 0	X144 = 0	 
X151 = 0	X152 = 548	X153 = 615	X154 = 248	 
X211 = 7	X212 = 0	X213 = 0	X214 = 22	 
X221 = 85	X222 = 85	X223 = 85	X224 = 0	 
X231 = 0	X232 = 0	X233 = 0	X234 = 285	 
X241 = 235	X242 = 235	X243 = 206	X244 = 0	 
X251 = 0	X252 = 0	X253 = 0	X254 = 3	 
X311 = 645	X312 = 650	X313 = 638	X314 = 663	 
X321 = 0	X322 = 0	X323 = 0	X324 = 0	 
X331 = 0	X332 = 0	X333 = 0	X334 = 0	 
X341 = 0	X342 = 0	X343 = 0	X344 = 0	 
X351 = 0	X352 = 0	X353 = 0	X354 = 3	 


In [13]:
for j in range(J):
    for t in range(T):
        print("y%d%d = %d"%(j+1, t+1, y.get((j,t)).X), end = "  \t")
    print(" ")

y11 = 1  	y12 = 1  	y13 = 1  	y14 = 1  	 
y21 = 1  	y22 = 1  	y23 = 1  	y24 = 1  	 
y31 = 0  	y32 = 0  	y33 = 0  	y34 = 1  	 
y41 = 1  	y42 = 1  	y43 = 1  	y44 = 0  	 
y51 = 0  	y52 = 1  	y53 = 1  	y54 = 1  	 


In [14]:
print("z1 = %f"%(z1_obj.getValue()))
print("z2 = %f"%(z2_obj.getValue()))
print("z3 = %f"%(z3_obj.getValue()))

z1 = 2895237.842075
z2 = 5965.677456
z3 = 6187.580089


In [17]:
model.setObjectiveN?

Signature:
model.setObjectiveN(
    expr,
    index,
    priority=0,
    weight=1.0,
    abstol=1e-06,
    reltol=0.0,
    name='',
)
Docstring:
ROUTINE:
  setObjectiveN(expr, index)

PURPOSE:
  Set the model objective equal to a LinExpr or QuadExpr

ARGUMENTS:
  expr:     The desired objective function.  The objective can be
            a linear expression (LinExpr) a variable (Var) or a constant.
            This routine will replace the 'ObjNVal' attribute on model variables
            with the corresponding values from the supplied expression for
            multi-objective 'index'
  index:    Identify which multi-objective to set
  priority: Set the ObjNPriority attribute for this multi-objective (default is zero)
  weight:   Set the ObjNWeight attribute for this multi-objective (default is 1.0)
  abstol:   Set the ObjNAbsTol  attribute for this multi-objective (default is 1e-6)
  reltol:   Set the ObjNRelTol  attribute for this multi-objective (default is zero)
  name:     multi